In [ ]:
from keras.datasets import reuters

(train_data, train_labels), (test_data, test_labels) = reuters.load_data(num_words = 10000)
#로이터 데이터셋 가져오고 각 데이터를 변수에 넣어줌

In [ ]:
#훈련용데이터와 검증용 데이터 몇개로 분류되었는지 출력.
print(len(train_data))
print(len(test_data))

print(train_data[1]) #2차원 배열로 들어가있음
print(train_labels[0]) #1차원 배열로 들어가있음.

In [ ]:
import numpy as np

def vectorize_sequences ( sequences, dimension = 10000):
    results = np.zeros((len(sequences), dimension)) #입력데이터만큼의 열을 만들고 각 행의 크기는 10000으로 설정, 값은 0으로 2차원배열만듬
    for i, sequence in enumerate(sequences) : #i는 0,1,2,...처럼 순서. sequnce는 입력데이터의 순서별 각 원소값
        results[i,sequence] = 1. #0번째 행의 0번째 원소값 열에 값에 실수형 1.00 삽입
    return results
# 이 함수를 통해 안에 값이 있으면 1, 값이 없으면 0으로 정규화를 시켜줬음

x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

#확인
print(len(x_train))
print(len(x_test))

print(len(x_train[0]))
print(x_train[0])
print(len(x_test[0]))
print(x_test[0])

In [ ]:
import keras
one_hot_train_labels = keras.utils.to_categorical(train_labels)
one_hot_test_labels = keras.utils.to_categorical(test_labels)

print(len(one_hot_train_labels)) #디멘션이 46임
print(len(one_hot_test_labels[0]))

train_data와 test_data는 파이썬 리스트의 넘파이 배열이기 때문에 to_categorical() 함수를 사용하지 못합니다. x_train과 x_test의 크기는 각각 (8982, 10000), (2246, 10000)이 되고 one_hot_train_labels와 one_hot_test_labels의 크기는 각각 (8982, 46), (2246, 46)이 됩니다.

In [ ]:
from keras import models
from keras  import layers

model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape = (10000,)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(46, activation='softmax')) # 각 입력 샘플에 대해서 46차원의 벡터를 출력 / 이 벡터의 각 원소(각 차원)는 각기 다른 출력 클래스가 인코딩된 것 / 확률 분포


• 마지막 Dense 층의 크기가 46입니다. 각 입력 샘플에 대해서 46차원의 벡터를 출력한다는 뜻입니다. 이 벡터의 각 원소(각 차원)는 각기 다른 출력 클래스가 인코딩된 것입니다.

• 마지막 층에 softmax 활성화 함수가 사용되었습니다. MNIST 예제에서 이런 방식을 보았습니다. 각 입력 샘플마다 46개의 출력 클래스에 대한 확률 분포를 출력합니다. 즉 46차원의 출력 벡터를 만들며 output[i]는 어떤 샘플이 클래스 i에 속할 확률입니다. 46개의 값을 모두 더하면 1이 됩니다.

In [ ]:
model.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])


이런 문제에 사용할 최선의 손실 함수는 categorical_crossentropy입니다. 이 함수는 두 확률 분포 사이의 거리를 측정합니다. 여기에서는 네트워크가 출력한 확률 분포와 진짜 레이블의 분포 사이의 거리입니다. 두 분포 사이의 거리를 최소화하면 진짜 레이블에 가능한 가까운 출력을 내도록 모델을 훈련하게 됩니다.

In [ ]:
x_val = x_train[:1000] #뒤에서 천개 때서 검증용 데이터로 사용
partial_x_train = x_train[1000:] #7982개.
y_val = one_hot_train_labels[:1000]
partial_y_train = one_hot_train_labels[1000:]

history = model.fit(partial_x_train, partial_y_train, epochs=20, batch_size=512, validation_data=(x_val, y_val))

In [ ]:
import matplotlib.pyplot as plt

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(loss) + 1)

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend() #하나의 그래프안에 그리기?

plt.show()

In [ ]:
plt.clf()

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

In [ ]:
model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(46, activation='softmax'))


model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(partial_x_train,
          partial_y_train,
          epochs=9,
          batch_size=512,
          validation_data=(x_val, y_val))
results = model.evaluate(x_test, one_hot_test_labels)

In [ ]:
print(results)

대략 78%의 정확도를 달성했습니다. 균형 잡힌 이진 분류 문제에서 완전히 무작위로 분류하면 50%의 정확도를 달성합니다. 이 문제는 불균형한 데이터셋을 사용하므로 무작위로 분류하면 18% 정도를 달성합니다. 여기에 비하면 이 결과는 꽤 좋은 편입니다.

>>> import copy
>>> test_labels_copy = copy.copy(test_labels)
>>> np.random.shuffle(test_labels_copy)
>>> hits_array = np.array(test_labels) == np.array(test_labels_copy)
>>> float(np.sum(hits_array)) / len(test_labels)
0.182546749777382